In [1]:
!pip install openai==0.28

In [ ]:
import openai
import pandas as pd

# OpenAI API 키 설정
openai.api_key = "your-api-key"

In [5]:
# 1. 데이터 로드
df = pd.read_csv("test.csv")  

# 2. 라벨 4에 해당하는 문장 필터링
label4_sentences = df[df["class"] == 4]["text"].tolist()
print(label4_sentences)

['이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 요즘 듣는 것도 들어봐 음 난 좀 별론데 좋을 줄 알았는데 아쉽네 내 취향은 아닌 듯 배고프다 밥이나 먹으러 가자 그래', '야 저기야 닭꼬치 트럭왔다 응 그러네 그치? 너도 먹고 싶지? 어?나도? 그래 너 세상에 닭꼬치 싫어하는 사람이 어딨어 그렇긴한데 배부른데 그래? 그럼 나만 먹고 올게 응. 뭐해?? 뭘? 돈줘야 먹고오지 내가 돈이 어딨냐 할머니랑 둘이 사는데? 응.?? 어 알겠어 얼른줘 미적거리지말고', '안경 안쓰고 키작고통통해보이는 분이었는데 다안경안씀  쌍꺼풀없었던것같고 아 단발인가 키작은사람들은 검정색 머리인데 없어가지고 단발인가 단발은 아니었던것같은데 막 나같은 장발도아님 모르겟다 중단발  영업직분중 하나인가 그럴수도', '아 취업하고 싶은데 누가 나 좀 안 데려가냐 아 그니까 나도 취업하고싶다 코로나때문에 미친거같아 진짜 너무 안 뽑아 그니까 동기들중에 취업한애들 아는애들 있어 뭐 지한이 삼성갔다그러고 그 여자애 이름 기억 안나는데 카카오 들어갔다던데 와 씨발 다들 잘갔네 부럽다 그니까 그냥 그럭저럭 학교생활 했는데 쫌 더 열씸히 할껄 아 난 나름 열심히해서 학점도 좋고한데 왜 안되냐 맨날 면접가서 떨어지니까 더 힘드네 너는 면접연습만 하면 될거같은데 나는 서류에서 광탈이야 아 그러게 면접연습 해야되는데 타고난게 말을 잘 못하는데 어쩌라는거야 그니까 공부도 잘해야되고 말도잘해야 취업되는 더러운세상 에휴 부모님한테도 죄송하고 얼른 취업해야되는데', '과장님 오늘 부장님 기분이 안 좋으신것 같네요 오늘 프레젠테이션 있었는데 사장님한테 엄청 까였나봐 아 그래요 오늘 조심해야겠네요 엉 알아서들 조심해 완전 저기압인거 같아 네 근데 오늘 저희 주간 회의 있어서 어차피 부딪힐거 같은데 그때까지 부장님 기분이 풀리길 바래야지 뭐 아 오늘 잘못하면 엄청 혼날꺼 같은데 그니까 준비 잘해 네 알겠습니다', '응응 점점 깊이 깊이 이야기가 들어갈수록 응응 맞아 마음이 좀 그럴수

In [6]:
group_size = 10
num_groups = 10

# 데이터가 충분한지 검사
if len(label4_sentences) < group_size * num_groups:
    raise ValueError("라벨 4의 예시 데이터가 충분하지 않습니다.")

all_generated_sentences = []

# 그룹별로 예시를 나누어 새 문장 생성
for i in range(num_groups):
    start_idx = i * group_size
    end_idx = start_idx + group_size
    group_examples = label4_sentences[start_idx:end_idx]

    # 프롬프트 구성: 10개의 예시를 기반으로 10개의 새로운 문장 생성 요청
    prompt = (
        "아래 예시와 유사한 문장을 생성해줘. "
        "다음 예시들은 참고용 문장이야. 아래 예시를 바탕으로 총 10개의 새로운 문장을 만들어줘. "
        "각 문장은 줄바꿈으로 구분해줘.\n\n"
    )
    for idx, ex in enumerate(group_examples, 1):
        prompt += f"예시 {idx}: {ex}\n"
    prompt += "\n새로운 문장 (각 줄마다 하나씩, 총 10개):"

    # ChatGPT API 호출
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=3000  # 필요에 따라 조정
    )
    
    generated_text = response.choices[0].message.content.strip()
    # 줄 단위로 분리하여 생성된 문장을 리스트에 추가
    sentences = [line.strip() for line in generated_text.split("\n") if line.strip()]
    all_generated_sentences.extend(sentences)

# 생성된 전체 문장 개수 출력
print("전체 생성된 문장 개수:", len(all_generated_sentences))

전체 생성된 문장 개수: 99


In [10]:
print(len(all_generated_sentences))

68


In [11]:
# 리스트를 DataFrame으로 변환합니다.
df_generated = pd.DataFrame(all_generated_sentences, columns=["generated_sentence"])

# DataFrame을 CSV 파일로 저장합니다.
df_generated.to_csv("generated_sentences.csv", index=False)

print("CSV 파일로 저장되었습니다: generated_sentences.csv")

CSV 파일로 저장되었습니다: generated_sentences.csv
